## Catalan Numbers and RNA secondary structure

### The Human Knot 
- チームで隣にいない人と手をつなぎ、それをほどくというアイスブレイクがhuman knot
- 偶数人で、どの人も他のつながれた手と交差するように手をつなぐとき、何通り存在するか？という組み合わせ問題につながる
- これはRNA folding の優れたアナロジーである
- 二つのステムループ構造を含むRNAの二次構造はpseudo-knotと呼ばれる
- なぜなら、実際には結び目ではないから
- [Wikipedia](https://ja.wikipedia.org/wiki/%E3%82%B7%E3%83%A5%E3%83%BC%E3%83%89%E3%83%8E%E3%83%83%E3%83%88)
- pseudo-knot は　knot ではないが、これによってRNAは折りたたまれることがある
- (optional)pseudo knot は従来のDPベースの二次構造予測では、予測困難であった
  - なぜ？

### Problem
- edge 同士が交差していないならば、マッチングしているグラフは noncrossingである
- n 個のノードが円形に並ぶグラフを考えるとき、noncrossing なマッチングでは、edge {i,j}, {k,l}を考えるとき、四つのノードは i < k < j < l にはならない (i < k　とする)
- あり得るのは、 i < j < k < l, i < k < l < j のどちらか
- pseudoknotは、bonding graph では交差するが、直鎖で考えると交差しない
- bonding graph で non crossing なマッチングを求めると、pseudoknot以外の二次構造の組み合わせを推定できる (エネルギー的な問題はあれど)
- 完全マッチングは、ノードの数が偶数でないと成立しない ため、ノード数を K2nとする
- その時の noncrossing な完全マッチングの組み合わせの数をCnとする
- C0 = 1, C1 = 1 になる

- 各ノードに 1:2n の番号をつける
- 完全マッチが成立するためには、必ず偶数ノード-奇数ノードでエッジをつながないといけない
- 1 から エッジをつなぐと、相手は必ず 2k になる
- {1:2k} で　エッジを引いた場合、2k-2, 2n-2k 個のノードに分割される
-  その場合、両方の分画は独立なので、ある kの時、Ck-1 * Cn-k になる
-  k は 1:nまでの値を取りえるので、Cn = sum(Ck-1 * Cn-k for k in range(1,n+1)) となる
-  このCnの漸化式は、カタラン数と同じである

### Given
- AとU, CとGが同じ数存在するようなRNA配列、最大長300bp

### Return
- noncrossing perfect matching の組み合わせの数を、1,000,000 で割った余り

### 解法アイデア
- 手を繋げる相手に制約があるので、純粋なカタラン数ではない
- 区間dpを使う
- dp[l][r] == 区間[l:r]におけるnoncrossing perfect matching の数 と定義する
- l < k < r　の時、
- dp[l][r] = dp[l+1,k-1] * dp[k+1,r]
- 結局、短い区間からすべての区間のnon-crossing match を順に求めれば、dpテーブルが埋まる

In [3]:
seq = "AUAU"

In [ ]:
import numpy as np

def noncrossing_matchings(seq):
    n = len(seq)
    basepair_map = {"A":"U", "U":"A", "G":"C", "C":"G"}
    # 任意の区間のnoncrossing perfect matchingを求めたいので、n×nのdp matrixを作る
    dp = np.zeros((n,n), dtype = int)

    # 短い区間から順に埋めていけば長い区間は計算できる
    # 完全マッチになりえる区間長は2の倍数である
    for search_len in range(2,n+1,2):
        for l in range(0, n + 1 - search_len):
            r = l + search_len - 1
            # 区間 l-r をさらに分割する
            for k in range(l+1,r+1, 2):
                if seq[l] != basepair_map[seq[k]]:
                    continue
                # 空区間の場合は1とする(そうしないとすべて0になる)
                former = dp[l+1][k-1] if l+1 <= k-1 else 1
                latter = dp[k+1][r] if k+1 <= r else 1 
                dp[l][r] += former * latter
            # 数値が溢れるのを防ぐため、毎回 modする
            dp[l][r] = dp[l][r] % 1000000
    
    return dp[0][n-1]


In [5]:
conv = noncrossing_matchings(seq)
print(conv)

2
